# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 10 2022 2:26PM,248289,19,ADV80006375,"AdvaGen Pharma, Ltd",Released
1,Oct 10 2022 2:26PM,248289,19,ADV80006376,"AdvaGen Pharma, Ltd",Released
2,Oct 10 2022 2:26PM,248289,19,ADV80006377,"AdvaGen Pharma, Ltd",Released
3,Oct 10 2022 2:26PM,248289,19,ADV80006378,"AdvaGen Pharma, Ltd",Released
4,Oct 10 2022 2:26PM,248289,19,ADV80006379,"AdvaGen Pharma, Ltd",Released
5,Oct 10 2022 2:26PM,248289,19,ADV80006380,"AdvaGen Pharma, Ltd",Released
6,Oct 10 2022 1:38PM,248287,19,8628963,"Emersa Waterbox, LLC",Released
7,Oct 10 2022 1:29PM,248286,19,TR-RCVC-10102022,"GCH Granules USA, Inc.",Released
8,Oct 10 2022 1:24PM,248285,10,CTF0011444,"Citieffe, Inc.",Executing
9,Oct 10 2022 1:05PM,248283,10,Enova-10781,Emerginnova,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,248283,Executing,8
15,248285,Executing,1
16,248286,Released,1
17,248287,Released,1
18,248289,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248283,NaN,NaN,8.0,NaN
248285,NaN,NaN,1.0,NaN
248286,NaN,NaN,NaN,1.0
248287,NaN,NaN,NaN,1.0
248289,NaN,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248213,1.0,12.0,8.0,2.0
248228,0.0,0.0,0.0,11.0
248244,0.0,0.0,0.0,3.0
248245,0.0,21.0,17.0,1.0
248260,0.0,0.0,13.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248213,1.0,12,8,2
248228,0.0,0,0,11
248244,0.0,0,0,3
248245,0.0,21,17,1
248260,0.0,0,13,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,248213,1.0,12,8,2
1,248228,0.0,0,0,11
2,248244,0.0,0,0,3
3,248245,0.0,21,17,1
4,248260,0.0,0,13,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,248213,1.0,12,8,2
1,248228,0.0,,,11
2,248244,0.0,,,3
3,248245,0.0,21,17,1
4,248260,0.0,,13,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 10 2022 2:26PM,248289,19,"AdvaGen Pharma, Ltd"
6,Oct 10 2022 1:38PM,248287,19,"Emersa Waterbox, LLC"
7,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc."
8,Oct 10 2022 1:24PM,248285,10,"Citieffe, Inc."
9,Oct 10 2022 1:05PM,248283,10,Emerginnova
17,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC"
18,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc."
38,Oct 10 2022 10:48AM,248260,15,"Brookfield Pharmaceuticals, LLC"
65,Oct 10 2022 9:21AM,248245,10,Bio-PRF
104,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Oct 10 2022 2:26PM,248289,19,"AdvaGen Pharma, Ltd",0.0,,,6
1,Oct 10 2022 1:38PM,248287,19,"Emersa Waterbox, LLC",0.0,,,1
2,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc.",0.0,,,1
3,Oct 10 2022 1:24PM,248285,10,"Citieffe, Inc.",0.0,,1,
4,Oct 10 2022 1:05PM,248283,10,Emerginnova,0.0,,8,
5,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC",0.0,,,1
6,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc.",0.0,,5,15
7,Oct 10 2022 10:48AM,248260,15,"Brookfield Pharmaceuticals, LLC",0.0,,13,14
8,Oct 10 2022 9:21AM,248245,10,Bio-PRF,0.0,21,17,1
9,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",0.0,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 2:26PM,248289,19,"AdvaGen Pharma, Ltd",6,,
1,Oct 10 2022 1:38PM,248287,19,"Emersa Waterbox, LLC",1,,
2,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc.",1,,
3,Oct 10 2022 1:24PM,248285,10,"Citieffe, Inc.",,1,
4,Oct 10 2022 1:05PM,248283,10,Emerginnova,,8,
5,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC",1,,
6,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc.",15,5,
7,Oct 10 2022 10:48AM,248260,15,"Brookfield Pharmaceuticals, LLC",14,13,
8,Oct 10 2022 9:21AM,248245,10,Bio-PRF,1,17,21
9,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 2:26PM,248289,19,"AdvaGen Pharma, Ltd",6,,
1,Oct 10 2022 1:38PM,248287,19,"Emersa Waterbox, LLC",1,,
2,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc.",1,,
3,Oct 10 2022 1:24PM,248285,10,"Citieffe, Inc.",,1,
4,Oct 10 2022 1:05PM,248283,10,Emerginnova,,8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 2:26PM,248289,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN
1,Oct 10 2022 1:38PM,248287,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Oct 10 2022 1:24PM,248285,10,"Citieffe, Inc.",NaN,1.0,NaN
4,Oct 10 2022 1:05PM,248283,10,Emerginnova,NaN,8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 2:26PM,248289,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0
1,Oct 10 2022 1:38PM,248287,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Oct 10 2022 1:24PM,248285,10,"Citieffe, Inc.",0.0,1.0,0.0
4,Oct 10 2022 1:05PM,248283,10,Emerginnova,0.0,8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1489498,17.0,34.0,33.0
15,744800,30.0,18.0,0.0
19,744862,8.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1489498,17.0,34.0,33.0
1,15,744800,30.0,18.0,0.0
2,19,744862,8.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,17.0,34.0,33.0
1,15,30.0,18.0,0.0
2,19,8.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,15,Released,30.0
2,19,Released,8.0
3,10,Executing,34.0
4,15,Executing,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19
Status,,,
Completed,33.0,0.0,0.0
Executing,34.0,18.0,0.0
Released,17.0,30.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19
0,Completed,33.0,0.0,0.0
1,Executing,34.0,18.0,0.0
2,Released,17.0,30.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19
0,Completed,33.0,0.0,0.0
1,Executing,34.0,18.0,0.0
2,Released,17.0,30.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()